In [5]:
import numpy as np
from skvideo.io import vwrite, vread


def draw_keypoints(
		frame,
        frame_keypoints,
		h: int,
		w: int,
        size: int = 5,
        color: int | None = None):
	'''Draw keypoints in the frame'''
	fr = frame.copy()
	for x, y, z in frame_keypoints:
		# check if x or y are masked
		if np.ma.is_masked(x) or np.ma.is_masked(y) or np.isnan(x) or np.isnan(y):
			# print(x.data, y.data)
			continue
		else:
			x = int(x*w)
			y = int(y*h)
			fr[y-size:y+size:,x-size:x+size] = color if color is not None else 255-fr[y-size:y+size:,x-size:x+size]
	return fr

In [20]:
# added to fix the np version issue with skvideo
np.float = np.float64
np.int = np.int_

name = "health1_signer3_rep1_sentences_sentences0005"
keys = np.load(f"/mnt/data3/slt-datasets-3/GSL/GSL_mirror/poses/{name}.npy")
video = vread(f"/mnt/data3/slt-datasets-3/GSL/GSL_mirror/videos/{name}.mp4")

h, w = video.shape[1], video.shape[2]
for i_frame in range(keys.shape[0]):
	for i_person in range(keys.shape[1]):
		video[i_frame] = draw_keypoints(video[i_frame], keys[i_frame][i_person], h, w, size=2)
vwrite(f"{name}.mp4", video)

In [21]:
from IPython.display import Video

Video(f"{name}.mp4")

In [15]:
# for all npy files in /mnt/data3/slt-datasets-3/GSL/GSL_mirror/poses add an extra second dimension
import os
from tqdm import tqdm

path = "/mnt/data3/slt-datasets-3/GSL/GSL_mirror/poses"
for file in tqdm(os.listdir(path)):
	if file.endswith(".npy"):
		keys = np.load(f"{path}/{file}")
		keys = np.expand_dims(keys, axis=1)
		np.save(f"{path}/{file}", keys)

100%|██████████| 10295/10295 [06:21<00:00, 26.96it/s]
